In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
PATH = 'C:/Users/EunJin/Desktop/dacon/LG AI/'

In [2]:
def make_df(train_err, quality):
  #train_err  = pd.read_csv(PATH+'train_err_data.csv')
  train_err['year'] = train_err['time'].astype(str).apply(lambda x: int(x[:4]))
  train_err['month'] = train_err['time'].astype(str).apply(lambda x: int(x[4:6]))
  train_err['day'] = train_err['time'].astype(str).apply(lambda x: int(x[6:8]))

  train_err['hour'] = train_err['time'].astype(str).apply(lambda x: int(x[8:10]))
  train_err['min'] = train_err['time'].astype(str).apply(lambda x: int(x[10:12]))
  train_err['sec'] = train_err['time'].astype(str).apply(lambda x: int(x[12:14]))
  train_err['date'] = train_err['time'].astype(str).apply(lambda x: int(x[4:8]))

  ###err_type
  pivot = pd.pivot_table(train_err, index = ['user_id'],columns = 'errtype', values = ['time'],aggfunc='count',fill_value=0)
  df_errtype = pd.DataFrame(pivot.to_records())

  df_errtype = df_errtype.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errtype")) 

  ###err_day
  pivot2 = pd.pivot_table(train_err, index = ['user_id'],columns = 'date', values = ['time'],aggfunc='count',fill_value=0)
  df_errday = pd.DataFrame(pivot2.to_records())

  df_errday = df_errday.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","day")) 

  for i in list(range(1,33)) :
          df_errday.iloc[:,i+1] = df_errday.iloc[:,i+1] + df_errday.iloc[:,i]

  df_result = pd.merge(df_errtype, df_errday, on='user_id', how='inner')

  ###quality
  #quality = pd.read_csv(PATH+'train_quality_data.csv')
  quality.fillna(0) #quality_5변수에 NA값이 있음



  #quality의 문자열변수,정수형변수 -> 실수형으로 바꿔주기
  cols = ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
          'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12']
  quality[cols] = quality[cols].replace({',':''}, regex= True) 
  quality.iloc[:,3:16] = quality.iloc[:,3:16].astype(float) #quality_n 변수들 실수형으로 바꿔주기

  #quality5만들기(quality의 일부 + -1의 개수 뽑아냄)

  #minusQ 리스트는 각 user_id별 -1의 빈도수가 얼마나 나오는지 알기위해서 만드는 리스트임
  #현재는 모든 quality 로그가 기록될때 -1인 퀄리티가 발생하면 minuQ리스트에 -1을 아니라면 0을 할당하는 식으로 새로운 열을 만듦
  minusQ = []
  for i in range(0,len(quality)): #데이터를 보면 quality 0,1,2,5,6,11 번째에서 동시에 -1이 발발하는것을 알 수 있었음.
    if (quality.iloc[i,3] == -1 or quality.iloc[i,4] == -1 or quality.iloc[i,5] == -1 or quality.iloc[i,8] == -1 or quality.iloc[i,9] == -1 or quality.iloc[i,14] == -1) :
    #if (quality.iloc[i,3] == -1 and quality.iloc[i,4] == -1 and quality.iloc[i,5] == -1 and quality.iloc[i,8] == -1 and quality.iloc[i,9] == -1 and quality.iloc[i,14] == -1) :
      minusQ.append(-1)
    else:
      minusQ.append(0)

  quality['minusQ'] = minusQ #minusQ 열 추가해주기



  #규칙을 찾아보면 quality는 무언가 누적합의 경우가 많음 그래서 가장 큰 숫자(max)를 대표값으로 지정하도록 코딩하였음
  #minusQ의 경우는 -1의 발생 빈도를 볼 것이기 때문에, 예외적으로 count를 볼 것이기 떄문에 합(sum)으로 코딩하였음

  def func(x):
      d = {}
      d['quality_0'] = x['quality_0'].max()
      d['quality_1'] = x['quality_1'].max()
      d['quality_2'] = x['quality_2'].max()
      d['quality_3'] = x['quality_3'].max()
      d['quality_4'] = x['quality_4'].max()
      d['quality_5'] = x['quality_5'].max()
      d['quality_6'] = x['quality_6'].max()
      d['quality_7'] = x['quality_7'].max()
      d['quality_8'] = x['quality_8'].max()
      d['quality_9'] = x['quality_9'].max()
      d['quality_10'] = x['quality_10'].max()
      d['quality_11'] = x['quality_11'].max()
      d['quality_12'] = x['quality_12'].max()
      d['minusQ'] = x['minusQ'].sum()
      return pd.Series(d, index= ['quality_0', 'quality_1', 'quality_2','quality_3', 'quality_4', 'quality_5', 
                                  'quality_6', 'quality_7','quality_8', 'quality_9', 'quality_10', 'quality_11', 'quality_12', 'minusQ'])

  quality5 = quality.groupby(['user_id','time']).apply(func)

  quality5 = quality5.groupby(['user_id']).sum()
  quality5 = quality5.drop(columns = ['quality_3', 'quality_4']) 

  df_result = pd.merge(df_result, quality5, left_on='user_id', right_on='user_id', how='left')
  df_result.fillna(0, inplace = True)

  ###errcode
  pivot3 = pd.pivot_table(train_err, index = ['user_id'],columns = 'errcode', values = ['time'],aggfunc='count',fill_value=0)
  df_errcode = pd.DataFrame(pivot3.to_records())

  df_errcode = df_errcode.rename(columns=lambda x: x.replace("(","").replace(",","").replace(")","").replace("'","").replace(" ","").replace("time","errcode_"))

  df_result = pd.merge(df_result, df_errcode, left_on='user_id', right_on='user_id', how='left')
  df_result.fillna(0, inplace = True)

  return df_result

In [3]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_quality = pd.read_csv(PATH+'test_quality_data.csv')
X_test = make_df(test_err, test_quality)

In [4]:
X_test.head()

,user_id,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,errtype42,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,day1203,day1204,day1205,day1209,day1211,day1214,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errcode_-1010,errcode_-269,errcode_-270,errcode_0,errcode_0001,errcode_1,errcode_100,errcode_10020,errcode_10029,errcode_10030,errcode_10035,errcode_10043,errcode_10046,errcode_10072,errcode_101,errcode_10158,errcode_10165,errcode_10168,errcode_102,errcode_10217,errcode_10237,errcode_10274,errcode_103,errcode_103063,errcode_10330,errcode_10331,errcode_10334,errcode_10335,errcode_10361,errcode_104,errcode_10400,errcode_10475,errcode_10487,errcode_104942,errcode_105,errcode_10500,errcode_10512,errcode_10522,errcode_10523,errcode_10527,errcode_10553,errcode_10558,errcode_10563,errcode_10564,errcode_10567,errcode_10576,errcode_10580,errcode_10581,errcode_10585,errcode_10591,errcode_10598,errcode_10599,errcode_106,errcode_10609,errcode_10626,errcode_10638,errcode_10651,errcode_10653,errcode_10655,errcode_10659,errcode_10668,errcode_10675,errcode_10680,errcode_10683,errcode_10685,errcode_10686,errcode_10695,errcode_10698,errcode_107,errcode_10711,errcode_10717,errcode_10730,errcode_10731,errcode_10734,errcode_10736,errcode_10743,errcode_10751,errcode_10753,errcode_10763,errcode_10774,errcode_10779,errcode_10784,errcode_10785,errcode_10788,errcode_10790,errcode_108,errcode_10804,errcode_10805,errcode_10806,errcode_10812,errcode_10817,errcode_10822,errcode_10825,errcode_10829,errcode_10830,errcode_10843,errcode_10845,errcode_10848,errcode_10853,errcode_10857,errcode_10860,errcode_10867,errcode_10872,errcode_10874,errcode_108761,errcode_10881,errcode_10890,errcode_10894,errcode_10899,errcode_109,errcode_10911,errcode_10913,errcode_10915,errcode_10920,errcode_10923,errcode_10930,errcode_10935,errcode_10944,errcode_10963,errcode_10966,errcode_10979,errcode_10984,errcode_10985,errcode_109851,errcode_10987,errcode_10991,errcode_10992,errcode_10993,errcode_110,errcode_11002,errcode_11006,errcode_11010,errcode_11011,errcode_11023,errcode_11027,errcode_11037,errcode_11044,errcode_11049,errcode_11067,errcode_11077,errcode_11085,errcode_11090,errcode_11100,errcode_11105,errcode_11116,errcode_11124,errcode_11138,errcode_11160,errcode_11161422,errcode_11226,errcode_11246,errcode_112485,errcode_11308,errcode_11336,errcode_113979,errcode_11398,errcode_11416,...,errcode_8479,errcode_8492,errcode_8497,errcode_85,errcode_8507,errcode_8517,errcode_8540,errcode_8543,errcode_8549,errcode_8554,errcode_8558,errcode_8562,errcode_8564,errcode_8565,errcode_8571,errcode_8572,errcode_8577,errcode_8579,errcode_8583,errcode_8595,errcode_8599,errcode_86,errcode_8603,errcode_8605,errcode_8622,errcode_8630,errcode_8642,errcode_8646,errcode_8663,errcode_8667,errcode_8671,errcode_8682,errcode_8683,errcode_8699,errcode_87,errcode_8708,errcode_8715,errcode_8718,errcode_8720,errcode_8723,errcode_8726,errcode_8730,errcode_8731,errcode_8737,errcode_87429,errcode_8743,errcode_8745,errcode_8754,errcode_8758,errcode_87621,errcode_8763,errcode_8771,errcode_8773,errcode_8784,errcode_88,errcode_8802,errcode_8831,errcode_8834,errcode_8839,errcode_8843,errcode_8848,errcode_8849,errcode_8852,errcode_8855,errcode_8857,errcode_8868,errcode_8881,errcode_8889,errcode_8899,errcode_89,errcode_8913,errcode_8916,errcode_8924,errcode_8932,errcode_8941,errcode_8942,

In [5]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
train_quality = pd.read_csv(PATH+'train_quality_data.csv')
X_train = make_df(train_err, train_quality)

In [6]:
X_train.head()

,user_id,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,errtype42,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errcode_-269,errcode_-270,errcode_0,errcode_0001,errcode_1,errcode_100,errcode_10005,errcode_10018,errcode_10043,errcode_10073,errcode_10080,errcode_101,errcode_10101,errcode_10149,errcode_10153,errcode_10155,errcode_10164,errcode_10190,errcode_102,errcode_10222,errcode_10226,errcode_10240,errcode_10247,errcode_10263,errcode_10267,errcode_10274,errcode_102789,errcode_10280,errcode_103,errcode_10305,errcode_10309,errcode_103400,errcode_10350,errcode_10377,errcode_10385,errcode_103924,errcode_104,errcode_10406,errcode_10412,errcode_10420,errcode_10433,errcode_105,errcode_10518,errcode_10552,errcode_10554,errcode_10557,errcode_10566,errcode_10568,errcode_105690,errcode_10573,errcode_10584,errcode_106,errcode_10612,errcode_10630,errcode_10670,errcode_10674,errcode_10676,errcode_10684,errcode_10689,errcode_10692,errcode_107,errcode_10715,errcode_10720,errcode_10721,errcode_10725,errcode_10732,errcode_10743,errcode_10749,errcode_10752,errcode_10760,errcode_10768,errcode_10777,errcode_10782,errcode_10792,errcode_108,errcode_10803,errcode_10807,errcode_10812,errcode_108133,errcode_10814,errcode_10819,errcode_10827,errcode_10830,errcode_10833,errcode_10838,errcode_10839,errcode_10841,errcode_10851,errcode_10852,errcode_10861,errcode_10881,errcode_10894,errcode_10898,errcode_1089908,errcode_109,errcode_10902,errcode_10907,errcode_10908,errcode_10933,errcode_10934,errcode_10938,errcode_10945,errcode_10949,errcode_10955,errcode_10961,errcode_10972,errcode_10980,errcode_10995,errcode_10997,errcode_110,errcode_11007,errcode_11014,errcode_11015,errcode_11033,errcode_11043,errcode_11063,errcode_11068,errcode_11070,errcode_11078,errcode_11087,errcode_11088,errcode_11094,errcode_11100,errcode_11108,errcode_11113,errcode_11146,errcode_11151,errcode_11197,errcode_11251,errcode_11259,errcode_112910,errcode_11295,errcode_113066,errcode_11324,errcode_11355,errcode_11396,errcode_11399,errcode_11434,errcode_11516,errcode_11534,errcode_11650,errcode_1166733,errcode_11674,errcode_11717,errcode_11725,errcode_1174404,errcode_117471,errcode_11751,errcode_11818,errcode_118518,errcode_11864,errcode_11886,errcode_11906,errcode_11942,errcode_11959,errcode_12,errcode_120256,errcode_12026,errcode_12093,errcode_12099,errcode_12103,errcode_12120,errcode_12125,...,errcode_8252,errcode_8255,errcode_82561,errcode_8257,errcode_8258,errcode_8260,errcode_8263,errcode_82694,errcode_8271,errcode_8278,errcode_82799,errcode_83,errcode_8310,errcode_8322,errcode_8339,errcode_83468,errcode_8350,errcode_8355,errcode_8356,errcode_8372,errcode_8389,errcode_8391,errcode_8397,errcode_8398,errcode_84,errcode_8408,errcode_8414,errcode_8442,errcode_8453,errcode_8459,errcode_8460,errcode_8462,errcode_8473,errcode_8488,errcode_8494,errcode_85,errcode_8515,errcode_8522,errcode_8533,errcode_8543,errcode_8549,errcode_85885,errcode_8592,errcode_86,errcode_8606,errcode_8610,errcode_8637,errcode_8641,errcode_8645,errcode_8664,errcode_8673,errcode_8674,errcode_8684,errcode_8699,errcode_87,errcode_8719,errcode_8731,errcode_8733,errcode_8738,errcode_8740,errcode_87418,errcode_87427,errcode_8755,errcode_8760,errcode_8764,errcode_8771,errcode_8793,errcode_88,errcode_8811,errcode_8819,errcode_8840,errcode_8843,errcode_8

In [7]:
# y 만들기
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)

problem[train_prob.user_id.unique()-10000] = 1 

In [8]:
train_x = X_train.drop(['user_id'], axis=1).to_numpy()
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 2891)
(15000,)


In [9]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'dart', #gbdt
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015,
                'leraning_rate': 0.05,
                'num_leaves' : 50,
                'max_depth': -1, #제한없이 분기(default:-1)
                'min_data_in_leaf': 40
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 5000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 4
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Warning] Unknown parameter: leraning_rate
[LightGBM] [Warning] Unknown parameter: leraning_rate
[LightGBM] [Info] Number of positive: 3633, number of negative: 8367
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029844 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17669
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 144
[LightGBM] [Warning] Unknown parameter: leraning_rate
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.302750 -> initscore=-0.834237
[LightGBM] [Info] Start training from score -0.834237
[20]	valid_0's auc: 0.812856	valid_0's pr_auc: 0.810146
[40]	valid_0's auc: 0.819535	valid_0's pr_auc: 0.815397
[60]	valid_0's auc: 0.819615	valid_0's pr_auc: 0.814966
[80]	valid_0's auc: 0.818626	valid_0's pr_auc: 0.814761
[100]	valid_0's auc: 0.819285	valid_0's pr_auc: 0.814241
[120]	valid_0's auc: 0.817849	valid_0's pr_auc: 0.813652
[14

[2660]	valid_0's auc: 0.798847	valid_0's pr_auc: 0.800237
[2680]	valid_0's auc: 0.79874	valid_0's pr_auc: 0.799963
[2700]	valid_0's auc: 0.798305	valid_0's pr_auc: 0.799697
[2720]	valid_0's auc: 0.798547	valid_0's pr_auc: 0.799868
[2740]	valid_0's auc: 0.798448	valid_0's pr_auc: 0.799573
[2760]	valid_0's auc: 0.79809	valid_0's pr_auc: 0.799417
[2780]	valid_0's auc: 0.798429	valid_0's pr_auc: 0.799674
[2800]	valid_0's auc: 0.798783	valid_0's pr_auc: 0.799924
[2820]	valid_0's auc: 0.798859	valid_0's pr_auc: 0.800073
[2840]	valid_0's auc: 0.799088	valid_0's pr_auc: 0.800622
[2860]	valid_0's auc: 0.79917	valid_0's pr_auc: 0.801041
[2880]	valid_0's auc: 0.799024	valid_0's pr_auc: 0.800933
[2900]	valid_0's auc: 0.798896	valid_0's pr_auc: 0.800737
[2920]	valid_0's auc: 0.798985	valid_0's pr_auc: 0.800735
[2940]	valid_0's auc: 0.799021	valid_0's pr_auc: 0.800801
[2960]	valid_0's auc: 0.79852	valid_0's pr_auc: 0.800461
[2980]	valid_0's auc: 0.799096	valid_0's pr_auc: 0.800905
[3000]	valid_0's a

[260]	valid_0's auc: 0.824479	valid_0's pr_auc: 0.367023
[280]	valid_0's auc: 0.824015	valid_0's pr_auc: 0.367994
[300]	valid_0's auc: 0.824535	valid_0's pr_auc: 0.375172
[320]	valid_0's auc: 0.82576	valid_0's pr_auc: 0.386731
[340]	valid_0's auc: 0.825976	valid_0's pr_auc: 0.387944
[360]	valid_0's auc: 0.824813	valid_0's pr_auc: 0.393314
[380]	valid_0's auc: 0.824262	valid_0's pr_auc: 0.387768
[400]	valid_0's auc: 0.824566	valid_0's pr_auc: 0.377718
[420]	valid_0's auc: 0.826406	valid_0's pr_auc: 0.381804
[440]	valid_0's auc: 0.824942	valid_0's pr_auc: 0.379767
[460]	valid_0's auc: 0.826013	valid_0's pr_auc: 0.366336
[480]	valid_0's auc: 0.826835	valid_0's pr_auc: 0.379231
[500]	valid_0's auc: 0.825304	valid_0's pr_auc: 0.389312
[520]	valid_0's auc: 0.825006	valid_0's pr_auc: 0.387592
[540]	valid_0's auc: 0.824667	valid_0's pr_auc: 0.386114
[560]	valid_0's auc: 0.823357	valid_0's pr_auc: 0.382818
[580]	valid_0's auc: 0.823079	valid_0's pr_auc: 0.390098
[600]	valid_0's auc: 0.81973	val

[3120]	valid_0's auc: 0.815321	valid_0's pr_auc: 0.358355
[3140]	valid_0's auc: 0.814928	valid_0's pr_auc: 0.358405
[3160]	valid_0's auc: 0.814393	valid_0's pr_auc: 0.356964
[3180]	valid_0's auc: 0.814525	valid_0's pr_auc: 0.356857
[3200]	valid_0's auc: 0.814667	valid_0's pr_auc: 0.356725
[3220]	valid_0's auc: 0.814344	valid_0's pr_auc: 0.356715
[3240]	valid_0's auc: 0.813546	valid_0's pr_auc: 0.357416
[3260]	valid_0's auc: 0.814546	valid_0's pr_auc: 0.358705
[3280]	valid_0's auc: 0.814472	valid_0's pr_auc: 0.358277
[3300]	valid_0's auc: 0.814622	valid_0's pr_auc: 0.358449
[3320]	valid_0's auc: 0.815187	valid_0's pr_auc: 0.358552
[3340]	valid_0's auc: 0.814548	valid_0's pr_auc: 0.360862
[3360]	valid_0's auc: 0.815323	valid_0's pr_auc: 0.361793
[3380]	valid_0's auc: 0.814677	valid_0's pr_auc: 0.360427
[3400]	valid_0's auc: 0.814227	valid_0's pr_auc: 0.360338
[3420]	valid_0's auc: 0.81418	valid_0's pr_auc: 0.361175
[3440]	valid_0's auc: 0.815084	valid_0's pr_auc: 0.361706
[3460]	valid_0'

[740]	valid_0's auc: 0.780373	valid_0's pr_auc: 0.412244
[760]	valid_0's auc: 0.78131	valid_0's pr_auc: 0.414491
[780]	valid_0's auc: 0.77989	valid_0's pr_auc: 0.412845
[800]	valid_0's auc: 0.780069	valid_0's pr_auc: 0.415519
[820]	valid_0's auc: 0.780563	valid_0's pr_auc: 0.417986
[840]	valid_0's auc: 0.781158	valid_0's pr_auc: 0.419691
[860]	valid_0's auc: 0.780446	valid_0's pr_auc: 0.422244
[880]	valid_0's auc: 0.779294	valid_0's pr_auc: 0.421911
[900]	valid_0's auc: 0.778464	valid_0's pr_auc: 0.419898
[920]	valid_0's auc: 0.778641	valid_0's pr_auc: 0.420789
[940]	valid_0's auc: 0.777167	valid_0's pr_auc: 0.421336
[960]	valid_0's auc: 0.778193	valid_0's pr_auc: 0.422649
[980]	valid_0's auc: 0.777178	valid_0's pr_auc: 0.423321
[1000]	valid_0's auc: 0.777192	valid_0's pr_auc: 0.423825
[1020]	valid_0's auc: 0.777163	valid_0's pr_auc: 0.42239
[1040]	valid_0's auc: 0.776254	valid_0's pr_auc: 0.421873
[1060]	valid_0's auc: 0.775534	valid_0's pr_auc: 0.421886
[1080]	valid_0's auc: 0.77575	

[3600]	valid_0's auc: 0.76895	valid_0's pr_auc: 0.404364
[3620]	valid_0's auc: 0.768486	valid_0's pr_auc: 0.404137
[3640]	valid_0's auc: 0.768314	valid_0's pr_auc: 0.403767
[3660]	valid_0's auc: 0.768717	valid_0's pr_auc: 0.4034
[3680]	valid_0's auc: 0.769187	valid_0's pr_auc: 0.404052
[3700]	valid_0's auc: 0.769119	valid_0's pr_auc: 0.404818
[3720]	valid_0's auc: 0.769659	valid_0's pr_auc: 0.404022
[3740]	valid_0's auc: 0.76939	valid_0's pr_auc: 0.404311
[3760]	valid_0's auc: 0.769454	valid_0's pr_auc: 0.40395
[3780]	valid_0's auc: 0.769327	valid_0's pr_auc: 0.403397
[3800]	valid_0's auc: 0.769519	valid_0's pr_auc: 0.401816
[3820]	valid_0's auc: 0.76917	valid_0's pr_auc: 0.399237
[3840]	valid_0's auc: 0.769227	valid_0's pr_auc: 0.400402
[3860]	valid_0's auc: 0.769096	valid_0's pr_auc: 0.401216
[3880]	valid_0's auc: 0.769065	valid_0's pr_auc: 0.401486
[3900]	valid_0's auc: 0.768982	valid_0's pr_auc: 0.400731
[3920]	valid_0's auc: 0.768465	valid_0's pr_auc: 0.40085
[3940]	valid_0's auc:

[1220]	valid_0's auc: 0.813603	valid_0's pr_auc: 0.541786
[1240]	valid_0's auc: 0.814173	valid_0's pr_auc: 0.541901
[1260]	valid_0's auc: 0.813884	valid_0's pr_auc: 0.542402
[1280]	valid_0's auc: 0.813453	valid_0's pr_auc: 0.543169
[1300]	valid_0's auc: 0.813785	valid_0's pr_auc: 0.543973
[1320]	valid_0's auc: 0.813391	valid_0's pr_auc: 0.54452
[1340]	valid_0's auc: 0.813778	valid_0's pr_auc: 0.544677
[1360]	valid_0's auc: 0.813822	valid_0's pr_auc: 0.5443
[1380]	valid_0's auc: 0.813611	valid_0's pr_auc: 0.544085
[1400]	valid_0's auc: 0.813503	valid_0's pr_auc: 0.544112
[1420]	valid_0's auc: 0.812223	valid_0's pr_auc: 0.543465
[1440]	valid_0's auc: 0.812237	valid_0's pr_auc: 0.542834
[1460]	valid_0's auc: 0.811666	valid_0's pr_auc: 0.542901
[1480]	valid_0's auc: 0.810987	valid_0's pr_auc: 0.541214
[1500]	valid_0's auc: 0.81085	valid_0's pr_auc: 0.540709
[1520]	valid_0's auc: 0.810606	valid_0's pr_auc: 0.540147
[1540]	valid_0's auc: 0.810491	valid_0's pr_auc: 0.539117
[1560]	valid_0's a

[4060]	valid_0's auc: 0.798341	valid_0's pr_auc: 0.524359
[4080]	valid_0's auc: 0.798475	valid_0's pr_auc: 0.52445
[4100]	valid_0's auc: 0.798528	valid_0's pr_auc: 0.524778
[4120]	valid_0's auc: 0.798226	valid_0's pr_auc: 0.523761
[4140]	valid_0's auc: 0.798141	valid_0's pr_auc: 0.523378
[4160]	valid_0's auc: 0.797862	valid_0's pr_auc: 0.521953
[4180]	valid_0's auc: 0.79799	valid_0's pr_auc: 0.522419
[4200]	valid_0's auc: 0.797751	valid_0's pr_auc: 0.521709
[4220]	valid_0's auc: 0.798029	valid_0's pr_auc: 0.520278
[4240]	valid_0's auc: 0.798046	valid_0's pr_auc: 0.520178
[4260]	valid_0's auc: 0.797978	valid_0's pr_auc: 0.520761
[4280]	valid_0's auc: 0.797878	valid_0's pr_auc: 0.521915
[4300]	valid_0's auc: 0.797962	valid_0's pr_auc: 0.52047
[4320]	valid_0's auc: 0.797986	valid_0's pr_auc: 0.521368
[4340]	valid_0's auc: 0.797907	valid_0's pr_auc: 0.521105
[4360]	valid_0's auc: 0.79804	valid_0's pr_auc: 0.520166
[4380]	valid_0's auc: 0.797898	valid_0's pr_auc: 0.51875
[4400]	valid_0's au

[1680]	valid_0's auc: 0.791232	valid_0's pr_auc: 0.97359
[1700]	valid_0's auc: 0.79144	valid_0's pr_auc: 0.973572
[1720]	valid_0's auc: 0.790835	valid_0's pr_auc: 0.973494
[1740]	valid_0's auc: 0.790978	valid_0's pr_auc: 0.973553
[1760]	valid_0's auc: 0.789503	valid_0's pr_auc: 0.973364
[1780]	valid_0's auc: 0.789332	valid_0's pr_auc: 0.97336
[1800]	valid_0's auc: 0.788503	valid_0's pr_auc: 0.973255
[1820]	valid_0's auc: 0.787583	valid_0's pr_auc: 0.973156
[1840]	valid_0's auc: 0.787409	valid_0's pr_auc: 0.973155
[1860]	valid_0's auc: 0.787697	valid_0's pr_auc: 0.973205
[1880]	valid_0's auc: 0.787176	valid_0's pr_auc: 0.973151
[1900]	valid_0's auc: 0.787202	valid_0's pr_auc: 0.973187
[1920]	valid_0's auc: 0.787452	valid_0's pr_auc: 0.973207
[1940]	valid_0's auc: 0.787818	valid_0's pr_auc: 0.973277
[1960]	valid_0's auc: 0.787413	valid_0's pr_auc: 0.973272
[1980]	valid_0's auc: 0.786692	valid_0's pr_auc: 0.973165
[2000]	valid_0's auc: 0.786264	valid_0's pr_auc: 0.973077
[2020]	valid_0's 

[4520]	valid_0's auc: 0.782275	valid_0's pr_auc: 0.972429
[4540]	valid_0's auc: 0.781757	valid_0's pr_auc: 0.972351
[4560]	valid_0's auc: 0.781997	valid_0's pr_auc: 0.972356
[4580]	valid_0's auc: 0.782382	valid_0's pr_auc: 0.972419
[4600]	valid_0's auc: 0.782277	valid_0's pr_auc: 0.972401
[4620]	valid_0's auc: 0.782707	valid_0's pr_auc: 0.972437
[4640]	valid_0's auc: 0.782377	valid_0's pr_auc: 0.972402
[4660]	valid_0's auc: 0.782634	valid_0's pr_auc: 0.972443
[4680]	valid_0's auc: 0.783038	valid_0's pr_auc: 0.972529
[4700]	valid_0's auc: 0.782625	valid_0's pr_auc: 0.972451
[4720]	valid_0's auc: 0.782306	valid_0's pr_auc: 0.972419
[4740]	valid_0's auc: 0.782309	valid_0's pr_auc: 0.972437
[4760]	valid_0's auc: 0.78267	valid_0's pr_auc: 0.972468
[4780]	valid_0's auc: 0.783167	valid_0's pr_auc: 0.972543
[4800]	valid_0's auc: 0.783234	valid_0's pr_auc: 0.97255
[4820]	valid_0's auc: 0.783533	valid_0's pr_auc: 0.972582
[4840]	valid_0's auc: 0.783169	valid_0's pr_auc: 0.972565
[4860]	valid_0's

In [10]:
print(np.mean(auc_scores))

0.7913776486739914


In [11]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1014
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=333)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 10000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 50, 
                        early_stopping_rounds = 5
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 4017, number of negative: 7983
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029539 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17824
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 158
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.334750 -> initscore=-0.686779
[LightGBM] [Info] Start training from score -0.686779
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[17]	valid_0's auc: 0.804554	valid_0's pr_auc: 0.720478
[LightGBM] [Info] Number of positive: 3999, number of negative: 8001
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.020245 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17817
[LightGBM] [Info] Number of data points in the train set: 12000, number of used fea

In [12]:
print(np.mean(auc_scores))

0.8150476866883988


In [13]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [15]:
X_test.head(1)

,user_id,errtype1,errtype2,errtype3,errtype4,errtype5,errtype6,errtype7,errtype8,errtype9,errtype10,errtype11,errtype12,errtype13,errtype14,errtype15,errtype16,errtype17,errtype18,errtype19,errtype20,errtype21,errtype22,errtype23,errtype24,errtype25,errtype26,errtype27,errtype28,errtype30,errtype31,errtype32,errtype33,errtype34,errtype35,errtype36,errtype37,errtype38,errtype39,errtype40,errtype41,errtype42,day1031,day1101,day1102,day1103,day1104,day1105,day1106,day1107,day1108,day1109,day1110,day1111,day1112,day1113,day1114,day1115,day1116,day1117,day1118,day1119,day1120,day1121,day1122,day1123,day1124,day1125,day1126,day1127,day1128,day1129,day1130,day1201,day1202,day1203,day1204,day1205,day1209,day1211,day1214,quality_0,quality_1,quality_2,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12,minusQ,errcode_-1010,errcode_-269,errcode_-270,errcode_0,errcode_0001,errcode_1,errcode_100,errcode_10020,errcode_10029,errcode_10030,errcode_10035,errcode_10043,errcode_10046,errcode_10072,errcode_101,errcode_10158,errcode_10165,errcode_10168,errcode_102,errcode_10217,errcode_10237,errcode_10274,errcode_103,errcode_103063,errcode_10330,errcode_10331,errcode_10334,errcode_10335,errcode_10361,errcode_104,errcode_10400,errcode_10475,errcode_10487,errcode_104942,errcode_105,errcode_10500,errcode_10512,errcode_10522,errcode_10523,errcode_10527,errcode_10553,errcode_10558,errcode_10563,errcode_10564,errcode_10567,errcode_10576,errcode_10580,errcode_10581,errcode_10585,errcode_10591,errcode_10598,errcode_10599,errcode_106,errcode_10609,errcode_10626,errcode_10638,errcode_10651,errcode_10653,errcode_10655,errcode_10659,errcode_10668,errcode_10675,errcode_10680,errcode_10683,errcode_10685,errcode_10686,errcode_10695,errcode_10698,errcode_107,errcode_10711,errcode_10717,errcode_10730,errcode_10731,errcode_10734,errcode_10736,errcode_10743,errcode_10751,errcode_10753,errcode_10763,errcode_10774,errcode_10779,errcode_10784,errcode_10785,errcode_10788,errcode_10790,errcode_108,errcode_10804,errcode_10805,errcode_10806,errcode_10812,errcode_10817,errcode_10822,errcode_10825,errcode_10829,errcode_10830,errcode_10843,errcode_10845,errcode_10848,errcode_10853,errcode_10857,errcode_10860,errcode_10867,errcode_10872,errcode_10874,errcode_108761,errcode_10881,errcode_10890,errcode_10894,errcode_10899,errcode_109,errcode_10911,errcode_10913,errcode_10915,errcode_10920,errcode_10923,errcode_10930,errcode_10935,errcode_10944,errcode_10963,errcode_10966,errcode_10979,errcode_10984,errcode_10985,errcode_109851,errcode_10987,errcode_10991,errcode_10992,errcode_10993,errcode_110,errcode_11002,errcode_11006,errcode_11010,errcode_11011,errcode_11023,errcode_11027,errcode_11037,errcode_11044,errcode_11049,errcode_11067,errcode_11077,errcode_11085,errcode_11090,errcode_11100,errcode_11105,errcode_11116,errcode_11124,errcode_11138,errcode_11160,errcode_11161422,errcode_11226,errcode_11246,errcode_112485,errcode_11308,errcode_11336,errcode_113979,errcode_11398,errcode_11416,...,errcode_8479,errcode_8492,errcode_8497,errcode_85,errcode_8507,errcode_8517,errcode_8540,errcode_8543,errcode_8549,errcode_8554,errcode_8558,errcode_8562,errcode_8564,errcode_8565,errcode_8571,errcode_8572,errcode_8577,errcode_8579,errcode_8583,errcode_8595,errcode_8599,errcode_86,errcode_8603,errcode_8605,errcode_8622,errcode_8630,errcode_8642,errcode_8646,errcode_8663,errcode_8667,errcode_8671,errcode_8682,errcode_8683,errcode_8699,errcode_87,errcode_8708,errcode_8715,errcode_8718,errcode_8720,errcode_8723,errcode_8726,errcode_8730,errcode_8731,errcode_8737,errcode_87429,errcode_8743,errcode_8745,errcode_8754,errcode_8758,errcode_87621,errcode_8763,errcode_8771,errcode_8773,errcode_8784,errcode_88,errcode_8802,errcode_8831,errcode_8834,errcode_8839,errcode_8843,errcode_8848,errcode_8849,errcode_8852,errcode_8855,errcode_8857,errcode_8868,errcode_8881,errcode_8889,errcode_8899,errcode_89,errcode_8913,errcode_8916,errcode_8924,errcode_8932,errcode_8941,errcode_8942,

In [16]:
X_train.to_csv("X_train_ej.csv", index = False)

In [17]:
X_test.to_csv("X_test_ej.csv", index = False)